In [4]:
import os 
from dotenv import load_dotenv

from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Vectara
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

try:
    from langchain_together import Together
except:
    print('Using deprecated Together LLM. Please use langchain_together instead.')
    from langchain_community.llms import Together

MODEL_NAME = "mistralai/Mixtral-8x7B-Instruct-v0.1"

load_dotenv()

Using deprecated Together LLM. Please use langchain_together instead.


True

In [1]:
from langchain.prompts import PromptTemplate


In [5]:
vectara_customer_id = os.environ['VECTARA_CUSTOMER_ID']
vectara_corpus_id = os.environ['VECTARA_CORPUS_ID']
vectara_api_key = os.environ['VECTARA_API_KEY']

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

vectara = Vectara(vectara_customer_id=vectara_customer_id,
                      vectara_corpus_id=vectara_corpus_id,
                      vectara_api_key=vectara_api_key)


In [2]:
template = """
passage: You are a helpful assistant that understands BIM building documents and engineering disciplines, specifically related to {discipline}.
passage: You will analyze two sets of BIM document metadata comprising of filename, description, and discipline. One set of metadata was automatically generated by AI in English, and the other was annotated by a human in German with a letter prefix.
passage: It is expected that several sets of annotated metadata are incorrect. You will translate the annotated metadata into English and determine if the information is contextually similar between each other.
passage: If the metadata sets are somewhat similar, you will advise so. If they are too different, you will specify that the annotated metadata is an anomaly. An anomaly is defined as metadata that is significantly different from the automatically generated metadata or does not match the expected format.
passage: Here are some examples of correct and incorrect metadata sets:
passage: Correct metadata set:
passage: Filename: ISB-020-U3-W-H-01-B17012-001-000.pdf, Description: Novocon S demontiert und Stellantriebe montiert, passage: Discipline: H - Heizung
passage: Incorrect metadata set:
passage: Filename: incorrect_document.pdf
passage: Description: This document contains specifications for the plumbing system in the building.
passage: Discipline: Electrical
passage: {context}
query: {question}
"""
prompt = ChatPromptTemplate.from_template(template)



NameError: name 'ChatPromptTemplate' is not defined

In [51]:
retriever = vectara.as_retriever()

model = Together(model=MODEL_NAME,
        temperature=0.7,
        max_tokens=256,
        top_k=50,
        together_api_key=os.environ["TOGETHER_API_KEY"]
        )

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [36]:
example_metadata = {"filename":"BedienungsanleitungFlanschen-Schieber", 
                    "description":"User manual for Flanschen-Schieber (Flange valves) PN 16. It includes information about storage, handling, transport, safety instructions, usage, product description, mounting, demounting, and spare parts.",
                    "discipline":"plumbing"}
madeup_meta = {'name':'electrical_doc.pdf',
               'description': 'Electrical wiring scheme and specifications for a generator room',
               'discipline':'ventilation'}

input_query = f'automatically_generated_metadata: {madeup_meta}, annotated_metadata: {example_metadata}'

output = chain.invoke(input_query)

In [52]:
example_metadata = {"filename":"BedienungsanleitungFlanschen-Schieber", 
                    "description":"User manual for Flanschen-Schieber (Flange valves) PN 16. It includes information about storage, handling, transport, safety instructions, usage, product description, mounting, demounting, and spare parts.",
                    "discipline":"plumbing"}
real_meta = {'name':'ISB-020-U3-W-S-01-B18003-001-020.pdf',
               'description': 'Schieber / Hawle / Schieber 4000 + Handrad 7800 DN100 Schutzraum',
               'discipline':'S - Sanitaer'}

input_query = f'automatically_generated_metadata: {real_meta}, annotated_metadata: {example_metadata}'

output = chain.invoke(input_query)	  	


In [53]:
output

"\nBased on the provided passage and query, the automatically generated metadata and the annotated metadata are different in terms of filename, description, and discipline. The automatically generated metadata refers to a specific document with the name 'ISB-020-U3-W-S-01-B18003-001-020.pdf', a description 'Schieber / Hawle / Schieber 4000 + Handrad 7800 DN100 Schutzraum', and a discipline 'S - Sanitaer'. On the other hand, the annotated metadata refers to a user manual for Flanschen-Schieber (Flange valves) PN 16 with a filename 'BedienungsanleitungFlanschen-Schieber', a description that includes information about storage, handling, transport, safety instructions, usage, product description, mounting, demounting, and spare parts, and a discipline 'plumbing'. Therefore, the annotated metadata can be considered an anomaly in this case."

In [56]:
summary_config = {"is_enabled": True, "max_results": 3, "response_lang": "eng"}
retriever = vectara.as_retriever(
    search_kwargs={"k": 3, "summary_config": summary_config}
)

In [58]:
def get_sources(documents):
    return documents[:-1]


def get_summary(documents):
    return documents[-1].page_content

In [60]:
query_str = f'Does this metadata set match the usual schema or is it an anomaly? metadata:{example_metadata}'
ans = (retriever | get_summary).invoke(query_str)
ans 



'The metadata set provided appears to be related to plumbing, specifically Flanschen-Schieber (Flange valves) PN 16. The description includes details about storage, handling, transport, safety instructions, usage, product description, mounting, demounting, and spare parts. The discipline associated with this metadata is plumbing [3]. The filenames and descriptions of various files related to plumbing and ventilation systems are listed in the search results, indicating a match with the usual schema for such metadata sets [2, 3].'

In [63]:
query_str = f"""Does this metadata set match the usual schema or is it an anomaly? metadata:
{real_meta}"""
ans = (retriever | get_summary).invoke(query_str)
ans 


'The metadata set provided with the name "ISB-020-U3-W-S-01-B18003-001-020.pdf," description "Schieber / Hawle / Schieber 4000 + Handrad 7800 DN100 Schutzraum," and discipline "S - Sanitaer" matches the usual schema as it follows the pattern of other entries such as "ISB-020-U3-W-S-01-B17012-011-000" with a similar structure and discipline [160]. The discipline "S - Sanitaer" is consistent with entries like "ISB-020-U3-W-S-01-B19009-001-020" that also belong to the Sanitaer discipline [161]. The description detailing specific items like "Schieber / Hawle / Schieber 4000 + Handrad 7800 DN100 Schutzraum" aligns with the content found in other entries under the same discipline [162]. The metadata set appears to be in line with the expected schema and does not seem to be an anomaly based on the available search results.'

In [65]:
query_str = f"""query: Does this metadata in either english or german set match the usual schema or is it an anomaly? metadata:
{madeup_meta}."""
ans = (retriever | get_summary).invoke(query_str)
ans 


"The metadata provided for 'electrical_doc.pdf' does not appear to match the usual schema for electrical documents. The discipline specified as 'ventilation' seems to be inconsistent with typical electrical document schemas [3]. The descriptions and names in the search results are more aligned with electrical systems rather than ventilation [10, 12, 13]. This indicates a potential anomaly in the metadata provided, suggesting a deviation from the expected schema for electrical documentation. It would be advisable to review the metadata to ensure accuracy and alignment with the appropriate discipline to avoid any confusion or misinterpretation of the document's content."